# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project 2: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **

It sounds like a classification problem where we are trying to predict 2 classes. The first class contains students who have high risk in terms of failing graduation. It's our target group. The second includes students we should not worry about. 


At the same time we could potentially tacle the project as a regression problem. Let's say we want to know not just whether a student requires our attention, but who needs immediate intervention and who could wait. In this case we could assign a score to students pick the ones with the highest scores first.


But I'll be solving the case as a classification problem. 


## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [8]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score
from IPython.display import YouTubeVideo

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [9]:
# TODO: Calculate number of students
n_students = student_data.shape[0]

# TODO: Calculate number of features
n_features = student_data.shape[1]-1 #minus 1 feature column

# TODO: Calculate passing students
n_passed = len(student_data[student_data['passed'] == 'yes'])

# TODO: Calculate failing students
n_failed = len(student_data[student_data['passed'] == 'no'])

# TODO: Calculate graduation rate
grad_rate = n_passed*100.0/n_students

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [10]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [11]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [12]:
# TODO: Import any additional functionality you may need here
from sklearn.cross_validation import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=num_test, random_state=30)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **

#### Logistic Regression


- Describe one real-world application in industry where the model can be applied.
 - Credit scoring is one of the problems where logistic regression could be applied. It's the most trivial example that could be easely found online. Here are just some mentions of LR <a href=http://www.statsoft.com/Textbook/Credit-Scoring>here</a> and <a href=http://wwwf.imperial.ac.uk/~bm508/teaching/retailfinance/Lecture2.pdf>here</a>. 
 - As we know the trained model won't be 100% accurate, so in real life we'll get an error. Basically for business that means we'll get creditors with debts. Banks could sell debt poftfolios (debt cases) to collection agencies. The debt collection agencies are the firms specializing in pursueing payments of debts owed by individuals or businesses. Collection agencies have own call-centers and they contact debtors via phone. There is a chance that the debtor could repay his debt even without court decision. But how likely? We could use logistic regression to estimate that, so we could focus our efforts on those people.
 

- What are the strengths of the model; when does it perform well? 
 - Logistic regression is a simple model that could be trained on datasets of all sizes. The model computational cost fuction is linear that makes it very effecient even for huge datasets. Logistic regression estimates a probability that could be easily tuned based on domain specific knowledge. 



- What are the weaknesses of the model; when does it perform poorly?
 - Variables should be independent. If there are any complex relationships between the data and the target variable the model could not be useful.
 - Logistic regression could underperform on missing data. 
 - The model is sensitive to outliers.


- What makes this model a good candidate for the problem, given what you know about the data?
 - Logistic regression is a go-to model for many simple classification problems. It's used as a baseline for comparison with other more complex models. 


#### Gaussian Naive Bayes
- Describe one real-world application in industry where the model can be applied.
 - <a href=https://en.wikipedia.org/wiki/Naive_Bayes_spam_filtering>Many spam filters </a>use Naive Bayes for classification whether the incomming email is spam or not.

- What are the strengths of the model; when does it perform well? 
 - Can be trained on few datapoints. Very fast. 
 - According to machine learning practitioners often works great in practice.

- What are the weaknesses of the model; when does it perform poorly?
 - The independence assumption. The model is based on applying Bayes’ theorem with the “naive” assumption of independence between every pair of features. In real life variables could be correlated, e.g. education level may correlate with income, but Naive Bayes assumes that these parameters are independent.

- What makes this model a good candidate for the problem, given what you know about the data?
 - The dataset is small. One of the advantages of NB is it can be trained on few datapoints. 
 - The independence assumption is a very strong dissadvantage here, but I want to make sure that the NB model wouldn't work in practice. There is an opinion that it's not that bad. 


#### Support Vector Machines (SVM)
- Describe one real-world application in industry where the model can be applied. 
 - Images classification can be performed using SVMs. There are some papers on this topic 
 (e.g. <a href=http://olivier.chapelle.cc/pub/tnn99.pdf> here</a>.

- What are the strengths of the model; when does it perform well? 
 - Captures non-linear interactions between variables thanks to the kernel trick which can create non-linear decision boundaries. 
 - Works on small data sets.
 - Outliers doesn't affect the result.

- What are the weaknesses of the model; when does it perform poorly?
 - Memory-intensive on many data points and features. 
 - Tuning is time consuming.
 - SVM does only binary classification. 
 
- What makes this model a good candidate for the problem, given what you know about the data?
 - The dataset is small, so the model should not be computationaly intensive.
 - The dataset has many features and SVM is able to deal with it.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [13]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    total_time = end - start
    print "Trained model in {:.4f} seconds".format(total_time)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    f1 = f1_score(target.values, y_pred, pos_label='yes')
    return f1
    

def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))


### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [14]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import tree


# TODO: Initialize the three models
clf_A = GaussianNB()
clf_B = LogisticRegression(random_state=30)
clf_C = SVC(random_state=30)
classifiers_list = [clf_A, clf_B, clf_C]

# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train[:300]
y_train_300 = y_train[:300]

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)
train_lists = [(X_train_100, y_train_100), (X_train_200, y_train_200), (X_train_300, y_train_300)]

table_formatting =[]
for clf in classifiers_list:
    for X, y in train_lists:
        table_formatting.append(train_predict(clf, X, y, X_test, y_test))
        print "\n"

Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0010 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 0.8092.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.5893.


Training a GaussianNB using a training set size of 200. . .
Trained model in 0.0010 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 0.7972.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.7258.


Training a GaussianNB using a training set size of 300. . .
Trained model in 0.0010 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 0.7794.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.7377.


Training a LogisticRegression using a training set size of 100. . .
Trained model in 0.0020 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 0.8649.
Made predictions in 0.0010 seconds.
F1 score for test set: 0.7752.


Training a LogisticRegression using a training set size of 200. 

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - GaussianNB **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
|*100*	|0.001	|0	|**0.8092**	|0.5893|
|*200*	|0	|0	|0.7972	|0.7258|
|*300*	|0.001	|0	|0.7794	|**0.7377**|





** Classifer 2 - LogisticRegression**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
|*100*	|0.001	|0	|**0.8649**	|0.7752|
|*200*	|0.004	|0	|0.8294	|**0.7883**|
|*300*	|0.003	|0	|0.8303	|0.7385|


** Classifer 3 - SVC **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
|*100*	|0.001	|0	|0.85	|**0.8129**|
|*200*	|0.003	|0	|0.8562	|0.8000|
|*300*	|0.007	|0	|**0.8577**	|0.8108|


Let's compare the models for 300 training cases.

| Training Set Size | Training Time | F1 Score (train) | F1 Score (test) |
| :---------------: | :--------------------: | :--------------: | :-------------: |
|GaussianNB	|**0.001**	|0.7794	|0.7377|
|LogisticRegression	|0.003	|0.8303	|0.7385|
|SVC	|0.007	|**0.8577**	|**0.8108**|

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

Gaussian Naive Bayes, Support Vector Machine and Logistic Regression have been applied for the given problem. 

The model which utilizes SVC has higher F1 scores for both training and testing examples which basically means it's much more accurate comparing to others. As a consequence we'll be able to identify the majority of those students who need help.The main dissadvantage of the model is its training time. Nevertheless it could be a problem only in case of millions students, so we are on the safe side.

Considering all the data I recommend using SVC as a classifier in this case.


### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **

Support Vector Machines is our model of choise. How does it work?

We want to separate students into two groups. The ones who need intervention and students that don't need intervention. We can utilize the information we already have - the data about 395 students. Fortunately we know a lot about each individual student including gender, his or her age, family size and even romantic relationship status. We also know if a student passed or failed. 

Than we apply an algorithm that uses the historical data and predicts whether the new student will pass or not. Therefore we'll understand if any given student needs intervention. According to our analysis the best algorithm for such a prediction is SVM (Support Vector Machine).  

SVM produces a boundary that separates students that passed and those who did not. 


<img src="https://i.imgsafe.org/c5a0212ad7.png" height=20% width=20% align="left">

Let's assume that we have the simpliest case. We create a chart where X axis represens hours of study and Y shows grades. Green dots show that these students passed the exam and the red ones that they didn't. We can draw a perfectly straign line (vector) which separates those two types of students, so we'll get the red(failed) and green(passed) zones. 

When a new students comes we'll be able to put his/her data on this graph and predict if he/she needs our intervention. 

Unfortunately the reality is not that simple so we need to take into account a) many features and b) not simple cases where we are not able to draw a straight line. 
<br> <br>  <br>  <br>  <br>  <br>  
 
Luckily SVM can overcome these issues. And here is a trick. We just add one more dimetion and 2D becomes 3D like this:
<img src="http://www.eric-kim.net/eric-kim-net/posts/1/imgs/data_2d_to_3d.png">
And then we use "a sheet of paper" to separate the points (left picture). After getting back to the 2D space we'll get our line (right picture).
<img src="http://www.eric-kim.net/eric-kim-net/posts/1/imgs/data_2d_to_3d_hyperplane.png">

We can easily imagine 2D and 3D which means we can understand only 2-3 parameters at time,  but computers can work with multiple dimetions. In our data we've got multiple variables (gender, age, family size, etc) and the machine can take them into account and predict what we want to know.  


### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [30]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn import grid_search
from sklearn import metrics

# TODO: Create the parameters list you wish to tune
parameters = [{
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': [0.001, 0.01, 0.1, 1, 10, 100]
 }]

# TODO: Initialize the classifier
clf = SVC()

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = metrics.make_scorer(metrics.f1_score, pos_label='yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = grid_search.GridSearchCV(clf, parameters, f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0020 seconds.
Tuned model has a training F1 score of 0.8590.
Made predictions in 0.0010 seconds.
Tuned model has a testing F1 score of 0.8169.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **
In the final exercise we tuned model, but F1 score for training and testing increased not significantly. 

| Training Set Size | Training Time | F1 Score (train) | F1 Score (test) |
| :---------------: | :--------------------: | :--------------: | :-------------: |
|SVC	|0.007	|0.8577	|0.8108|
|SVC tuned	|0.010	|**0.8590**	|**0.8169**|

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.